In [1]:
import shopify
import datetime
from datetime import date, timedelta
import pandas as pd
API_KEY = '3b53e8c31f54f969f0ec188c5a3e2b60'
PASSWORD = '5449dca275ad2dfd3b30c776f4a60acc'
SHOP_NAME = 'iriscosmetics'
SHARED_SECRET = '77131d150311bf079cbdfc6939838daf'
shop_url = "https://%s.myshopify.com/admin" % (SHOP_NAME)

shopify.ShopifyResource.set_user(API_KEY)
shopify.ShopifyResource.set_password(PASSWORD)
shopify.ShopifyResource.set_site(shop_url)
#The following code prints the amount of orders up in your shopify store in command prompt window
#shop = shopify.Shop.current()


In [20]:
SINCE_NUM_DAYS = 1
UNTIL_NUM_DAYS = 0
def date_from_today(n):
    return str(date.today() - timedelta(n))
orders = shopify.Order.find(created_at_min = date_from_today(SINCE_NUM_DAYS),
                           created_at_max = date_from_today(UNTIL_NUM_DAYS))

In [21]:
orders

[]

In [24]:
LAPOSTE_COLUMNS = ['Raison sociale', 'Civilité', 'Nom', 'Prénom', 'Identité / Service / Etage',
 'Bâtiment / Immeuble', 'N° et libellé de voie', 'Lieu-dit / BP', 'Code Postal', 'Ville',
 'Mobile', 'Téléphone', 'Email', 'Référence Destinataire']

def laposte_df(orders):
    laposte_df = pd.DataFrame(columns = LAPOSTE_COLUMNS)
    for order in orders:
        shipping_data = order.to_dict()['shipping_address']
        df = pd.DataFrame(columns = LAPOSTE_COLUMNS)
        df['Nom'] = [shipping_data['last_name'].replace('  ', ' ').strip()]
        df['Prénom'] = shipping_data['first_name'].replace('  ', ' ').strip()
        df['N° et libellé de voie'] = shipping_data['address1'].replace('  ', ' ').strip()
        df['Code Postal'] = shipping_data['zip']
        df['Ville'] = shipping_data['city'].strip()
        df['Mobile'] = shipping_data['phone']
        df['Email'] = order.to_dict()['email']
        laposte_df = pd.concat([laposte_df, df])
    laposte_df = laposte_df.reset_index(drop = True)
    laposte_df.index = [i+1 for i in range(len(laposte_df))]
    return laposte_df
        

In [23]:
import os
subdir = 'laposte_csv/'
yesterday = date_from_today(1)

if not os.path.exists(subdir):
    os.mkdir(subdir)
laposte_df(orders).to_csv(subdir + '/' + yesterday +'.csv', sep = ';')

In [25]:
laposte_df(orders)

,Raison sociale,Civilité,Nom,Prénom,Identité / Service / Etage,Bâtiment / Immeuble,N° et libellé de voie,Lieu-dit / BP,Code Postal,Ville,Mobile,Téléphone,Email,Référence Destinataire
1,NaN,NaN,Chopin,Mireille,NaN,NaN,77 Boulevard Pasteur,NaN,25300,Pontarlier,06 25 89 10 07,NaN,erchopin@laposte.net,NaN
2,NaN,NaN,Devillers,Valérie,NaN,NaN,14 Rue Léon Augereau,NaN,49280,La Tessoualle,,NaN,valeriedevillers1@gmail.com,NaN


In [48]:
import re
MAX_LENGTH = 36
adress_input = "je     suis un con et je te pisse à la   raie espece de fils de pute de ta grand mere tchoiiiin"
def adress_treatment(adress_input):
    adress_input = re.sub('\s+', ' ', adress_input).strip()
    if len(adress_input) >= MAX_LENGTH:
        adress_troncated = adress_input[:MAX_LENGTH].strip()
        adress_first_half = ''.join(adress_troncated[:-1]).strip()
        adress_second_half = adress_input.replace(adress_first_half, '').strip()
        return adress_first_half, adress_second_half
    else:
        return adress_input

In [50]:
adress_treatment(adress_input)[0]

'je suis un con et je te pisse à la'

In [ ]:
import re

re.sub('\s+', ' ', mystring).strip()